In [34]:
import pandas as pd

In [41]:
admissions = pd.read_csv("./ADMISSIONS.csv")
# print statements are commented to not show any data from MIMIC-III
# admissions.head()

In [42]:
admissions = admissions[["SUBJECT_ID", "HADM_ID", "ADMITTIME"]]

admissions = admissions.rename(columns={
    "SUBJECT_ID": "subject_id", 
    "HADM_ID": "hadm_id", 
    "ADMITTIME": "timestamp"
    })
# admissions

In [43]:
notes = pd.read_csv("./NOTEEVENTS.csv")
# notes.head()

C:\Users\eunic\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [44]:
notes = notes[["SUBJECT_ID", "HADM_ID", "CHARTTIME", "TEXT"]]
# notes.head()

In [45]:
notes_with_charttime = notes[~pd.isnull(notes['CHARTTIME'])]['CHARTTIME']
# notes_with_charttime

In [46]:
print("Number of rows in NOTEEVENTS.csv: ", notes.shape)
print("Number of rows in NOTEEVENTS.csv with non-emtpy CHARTTIME field", notes_with_charttime.shape)

Number of rows in NOTEEVENTS.csv:  (2083180, 4)
Number of rows in NOTEEVENTS.csv with non-emtpy CHARTTIME field (1766614,)


In [49]:
patients = pd.read_csv("./PATIENTS.csv")
# patients.head()['DOD']

In [50]:
# Building labels for mortality prediction
patients['label'] = (~pd.isnull(patients.head()['DOD'])).astype(int)

In [51]:
patients = patients[["SUBJECT_ID", "label"]]
patients = patients.rename(columns={"SUBJECT_ID": "subject_id"})
# patients.head()

In [52]:
labels = pd.merge(patients, admissions, on="subject_id")
labels = labels[["subject_id","hadm_id","timestamp","label"]]
labels['label'] = labels['label'].fillna(0)
labels['label'] = labels['label'].astype(int)
# labels

In [66]:
chronologies = pd.read_csv("../public_mm_lite/chronologies.csv")
test_chronologies = pd.read_csv("../public_mm_lite/test.chronologies.csv")
train_chronologies = pd.read_csv("../public_mm_lite/train.chronologies.csv")
devel_chronologies = pd.read_csv("../public_mm_lite/devel.chronologies.csv")
# chronologies

In [67]:
# test_labels = pd.merge(test_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])
# train_labels = pd.merge(train_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])
# devel_labels = pd.merge(devel_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])

# test_labels.groupby(["hadm_id"])['timestamp'].agg('min').reset_index(name="timestamp").to_csv("test.labels.csv", index=False)
# train_labels.groupby(["hadm_id"])['timestamp'].agg('min').reset_index(name="timestamp").to_csv("train.labels.csv", index=False)
# devel_labels.groupby(["hadm_id"])['timestamp'].agg('min').reset_index(name="timestamp").to_csv("devel.labels.csv", index=False)

In [71]:
# Prepare chronology files
test_chronologies = test_chronologies.groupby(["hadm_id"]).agg({'subject_id': 'min', 'timestamp': 'max', 'observations': lambda x: " ".join(x)}).reset_index("hadm_id")
test_chronologies = test_chronologies[['subject_id', 'hadm_id', 'timestamp', 'observations']]
test_chronologies.to_csv("test.chronologies.csv", index=False)

train_chronologies['observations'] = train_chronologies['observations'].apply(lambda x: str(x))
train_chronologies = train_chronologies.groupby(["hadm_id"]).agg({'subject_id': 'min', 'timestamp': 'max', 'observations': lambda x: " ".join(x)}).reset_index("hadm_id")
train_chronologies = train_chronologies[['subject_id', 'hadm_id', 'timestamp', 'observations']]
train_chronologies.to_csv("train.chronologies.csv", index=False)

devel_chronologies = devel_chronologies.groupby(["hadm_id"]).agg({'subject_id': 'min', 'timestamp': 'max', 'observations': lambda x: " ".join(x)}).reset_index("hadm_id")
devel_chronologies = devel_chronologies[['subject_id', 'hadm_id', 'timestamp', 'observations']]
devel_chronologies.to_csv("devel.chronologies.csv", index=False)

In [72]:
# Prepare admittimes files
test_admittimes = pd.merge(test_chronologies[["subject_id", "hadm_id"]], admissions, on=["subject_id", "hadm_id"])
test_admittimes.to_csv("test.admittimes.csv", index=False)

train_admittimes = pd.merge(train_chronologies[["subject_id", "hadm_id"]], admissions, on=["subject_id", "hadm_id"])
train_admittimes.to_csv("train.admittimes.csv", index=False)

devel_admittimes = pd.merge(devel_chronologies[["subject_id", "hadm_id"]], admissions, on=["subject_id", "hadm_id"])
devel_admittimes.to_csv("devel.admittimes.csv", index=False)

In [73]:
# Prepare labels files
test_labels = pd.merge(test_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])
test_labels.to_csv("test.labels.csv", index=False)

train_labels = pd.merge(train_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])
train_labels.to_csv("train.labels.csv", index=False)

devel_labels = pd.merge(devel_chronologies[["subject_id", "hadm_id"]], labels, on=["subject_id", "hadm_id"])
devel_labels.to_csv("devel.labels.csv", index=False)

In [77]:
print("Number of test chronologies:", test_chronologies.groupby(["hadm_id"]).count().shape[0])
print("Number of train chronologies:", train_chronologies.groupby(["hadm_id"]).count().shape[0])
print("Number of devel chronologies:", devel_chronologies.groupby(["hadm_id"]).count().shape[0])

Number of test chronologies: 33
Number of train chronologies: 57
Number of devel chronologies: 37
